Medical Appointment No Shows.  Medical appointments are time commitments doctors make with patients.  Let's build models that predict whether the next appointment is a show or no show! 
Expected Output:
Multiple accuracy measures resembling different criteria used for training your decision tree classifiers.
Multiple accuracy measures resembling a different number of estimators used for your random forest classifiers.
One printed confusion matrix for the best model.  Use KaggleV2-May-2016.csv for this project.


In [9]:
import pandas as pd
df = pd.read_csv("KaggleV2-May-2016.csv")
from sklearn.preprocessing import LabelEncoder


# Check for Null, then Drop PatientId colunn, as it provides us with no data. 
df = df.dropna()
df.drop('PatientId', axis=1, inplace=True)

# Encode the Gender, and Neighbourhood
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Neighbourhood'] = label_encoder.fit_transform(df['Neighbourhood'])

df.head(6)





,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,5642903,0,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,39,0,1,0,0,0,0,No
1,5642503,1,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,39,0,0,0,0,0,0,No
2,5642549,0,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,45,0,0,0,0,0,0,No
3,5642828,0,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,54,0,0,0,0,0,0,No
4,5642494,0,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,39,0,1,1,0,0,0,No
5,5626772,0,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,58,0,1,0,0,0,0,No
